## The third In-class-exercise (9/16/2020, 20 points in total)

The purpose of this exercise is to under users' information needs, then collect the data for analysis.

Question 1 (8 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

Research Question:
Classify an unrated movie review as either positive or negative.

For solving the above question, I would need movie reviews that are already rated. 
I will collect 500 reviews for the movie Avengers: Endgame (2019) along with their star ratings.

The IMDB site loads 25 movie reviews at a time. After that you need to click the 'Load More' button to get more reviews.
For this I used webdriver from Selenium. I created a while loop that clicks on the Load More button and waits 1/2 second 
for more reviews to load. If the site loads more reviews within 1/2 second then the loop again clicks the Load More button.
This process continues until the site takes more than 1/2 second to load the reviews. In that case, an exception arises, 
and we break the loop.

Now that we have all our reviews opened up, I use BeautifulSoup to scrape the data. Each movie review is contained in a 
class = 'lister-item-content'. Not all reviews have star ratings.
Therefore, for each container that has a star rating, we find the class = content, take the text in that class and remove 
'\n' from the left side of the text. Then we store it in a list of reviews. So, we have a list of reviews.
To get the star ratings, we find the class = 'ipl-ratings-bar', take the text that is stored in the span tag, remove '\n'
from both sides of the text, and append this text to our list of star_ratings. So, we have a list of star_ratings.

Last step is to create a dataframe with both reviews and star ratings. This data can now be used to train models that can 
automatically classify reviews.

'''

Question 2 (12 points): Write python code to collect 500 items of the data you plan to collect above.

In [40]:
# You code here (Please add comments in the code):

# Importing the necessary packages.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Specifying the URL from where we will scrape the data.
URL = "https://www.imdb.com/title/tt4154796/reviews?ref_=tt_urv"

# Installing a driver in order to work with Selenium on a dynamic website.
driver = webdriver.Chrome(r"C:\Raheyma\Important Stuff\Masters Work\INFO 5731 - Computational Methods for Information Systems\chromedriver.exe")
driver.get(URL)

# Creating a while loop to click on the 'Load More' button multiple times.
while True:
    try:
        # Find the 'Load More' button and click it.
        driver.find_element_by_css_selector("button#load-more-trigger").click()
        # Wait half second for the next 'Load More' to load and scrape the page.
        WebDriverWait(driver,0.5).until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".ipl-load-more__load-indicator")))
        soup = BeautifulSoup(driver.page_source, 'lxml')
    # If the 'Load More' button takes more than 1/2 second, break the loop.
    except Exception:break

# Each movie review is contained in a class = 'lister-item-content'. Find all and store in movie_containers.        
movie_containers = soup.find_all(class_ = 'lister-item-content')
# Create lists to store reviews and star ratings.
reviews = []
star_ratings = []

# For each review, find if it has a star rating and scrape the reviews and star ratings.
for container in movie_containers:
    if container.find(class_ = 'ipl-ratings-bar') is not None: # If star rating is present
        
        # Then find class = content which contains the text of the review.
        text_review = container.find(class_ = 'content')
        # Remove '\n' from left side and get the text of the review.
        review = text_review.get_text().lstrip()
        # Append each review text to the list of reviews.
        reviews.append(review)
        
        # Then find class = ipl-ratings-bar which contains the number of stars given by the reviewer.
        star_review = container.find(class_ = 'ipl-ratings-bar')
        # Remove '\n' from both sides and get the number of stars.
        stars = star_review.span.get_text().strip()
        # Append each star rating to the list of star ratings.
        star_ratings.append(stars)

# Create a dataframe with reviews and star ratings. 
d = {'review': reviews,'star rating': star_ratings}
movie_reviews = pd.DataFrame(d)
reviews_500 = movie_reviews.head(500)
reviews_500

,review,star rating
0,There is no way that I could describe my emoti...,10/10
1,"I have to say, my first reaction walking out o...",6/10
2,"After Avengers Infinity War, we waited for the...",10/10
3,"If you're going to watch this movie, avoid any...",10/10
4,Unpopular opinion.\nLet me start by saying tha...,6/10
...,...,...
495,1st hour and a half in I was ready for it toen...,4/10
496,But I never liked super hero movies to to begi...,4/10
497,"Quite simply, this is the most overrated film ...",1/10
498,"I thought the Infinity Wars was bad, but this ...",2/10
